# Example Optimiser

In [44]:
import os 
import sys
import json
sys.path.append(os.path.abspath(".."))

from velopix_wrappers.parameter_optimisers import optimiserBase
from velopix_wrappers.velopix_pipeline import TrackFollowingPipeline, GraphDFSPipeline, SearchByTripletTriePipeline
from typing import Any, Dict
import random
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C


## Implement the optimiser child class

In [ ]:
from typing import Any, Dict
import random
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

class BayesianOptimiser(optimiserBase):
    def __init__(self, learning_rate, max_iterations=100, target_score=0.3):
        super().__init__()
        self.learning_rate = learning_rate
        self.max_iterations = max_iterations
        self.target_score = target_score
        self.best_score = float("inf")
        self.current_iteration = 0

        # To store previous evaluations
        self.X = []  # Parameter sets (input)
        self.Y = []  # Objective function results (output)

        # Gaussian Process initialization
        kernel = C(1.0, (1e-4, 1e1)) * RBF(1.0, (1e-4, 1e1))
        self.gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

    def init(self) -> Dict[str, Any]:
        """
        Initializes the optimization process by setting an initial parameter map.
        """
        pMap = self._algorithm.get_config()  # Get a copy of the parameter schema

        initial_param_set = {}

        for key, (expected_type, _) in pMap.items():
            if expected_type == float:
                initial_param_set[key] = random.uniform(0, 1)  # Random float between 0 and 1
            elif expected_type == int:
                initial_param_set[key] = random.randint(0, 10)  # Random integer between 0 and 10
            elif expected_type == bool:
                initial_param_set[key] = random.choice([True, False])  # Random boolean
            elif expected_type == list:
                initial_param_set[key] = []  # Assign an empty list (or populate it if needed)

        # Evaluate the initial point
        self.X.append(list(initial_param_set.values()))
        self.Y.append(self.objective_func(initial_param_set))

        print(f"Initial parameter set: {initial_param_set}")

        return initial_param_set

    def next(self) -> Dict[str, Any]:
        """
        Uses Bayesian Optimization to generate the next parameter map by predicting the next best set.
        """
        if len(self.X) > 1:
            # Fit Gaussian Process model
            self.gpr.fit(self.X, self.Y)

        # Generate the next set of parameters using Bayesian Optimization
        next_param_set = self._predict_next()

        self.X.append(list(next_param_set.values()))  # Add the new point to X
        self.Y.append(self.objective_func(next_param_set))  # Evaluate the next point and add to Y

        return next_param_set

    def _predict_next(self) -> Dict[str, Any]:
        """
        Predicts the next parameter set using the Gaussian Process model.
        """
        # For simplicity, we'll randomly sample a set of points to evaluate and pick the one with the best acquisition value.
        pMap = self._algorithm.get_config()
        param_space = []

        for key, (expected_type, _) in pMap.items():
            low, high = self._algorithm._bounds().get(key)
            if expected_type == float:
                # Sample a random float within the given bounds
                param_space.append(np.random.uniform(low, high, 10))  # 10 random candidates between low and high
            elif expected_type == int:
                # Sample a random integer within the given bounds
                param_space.append(np.random.randint(low, high + 1, 10))  # 10 random integers between low and high
            elif expected_type == bool:
                # Sample random booleans as before (since there are no bounds for booleans)
                param_space.append(np.random.choice([True, False], size=10))  # 10 random booleans
            elif expected_type == list:
                # For lists, sample randomly if there are bounds
                # This part may depend on how the bounds for lists are defined
                param_space.append([np.random.randint(0, 10, size=np.random.randint(1, 5)) for _ in range(10)])  # 10 random lists

        print(f"temp param space: {param_space}")

        #TODO: Save best parameter set to self.best_config(?)

        # Now you can use these candidates to query the acquisition function (simplified)
        best_candidate = None
        best_acquisition_value = float('inf')
        
        for candidate in self._generate_candidates(param_space):
            X_candidate = np.array(candidate).reshape(1, -1)
            acquisition_value = self._acquisition_function(X_candidate)
            if acquisition_value < best_acquisition_value:
                best_candidate = candidate
                best_acquisition_value = acquisition_value

        # Convert the best candidate back into the parameter set format
        pMap = self._algorithm.get_config()
        next_param_set = {key: value for key, value in zip(pMap.keys(), best_candidate)}

        print(f"Next parameter set: {next_param_set}")

        return next_param_set

    def _generate_candidates(self, param_space):
        """
        Generates candidates from the parameter space.
        """
        # In this case, we're just randomly sampling from the parameter space for simplicity
        candidates = list(np.array(np.meshgrid(*param_space)).T.reshape(-1, len(param_space)))
        return candidates

    def _acquisition_function(self, X_candidate):
        """
        Acquisition function to guide the optimization.
        For simplicity, using a simple negative expected improvement here.
        """
        mean, std = self.gpr.predict(X_candidate, return_std=True)
        return -mean  # Expected improvement simplification: pick the most uncertain area

    def objective_func(self, param_set: Dict[str, Any]) -> float:
        """
        Converts the results of an experiment into a numeric score.
        In this example, we simulate a loss function that we aim to minimize.
        """
        # Fake evaluation function
        # Use the actual parameters and compute the score from the experiment
        return abs(self.learning_rate - 0.05) + random.uniform(0, 0.01)

    def is_finished(self) -> bool:
        """
        Determines if the optimization process is finished.
        In this case, it stops after `max_iterations` iterations or the target score is reached.
        """
        return self.best_score < self.target_score or self.current_iteration >= self.max_iterations


**Load event data**

In [46]:
events = []
n_files = 100

for i in range(0, n_files):
    if i == 51:
        """
        There's an issue with event 51 -> module_prefix_sum contains value 79 twice resulting in and indexing error when loading the event
        """
        print(f"Skipping problematic file: velo_event_{i}.json")
    else:    
        print(f"Loading file: velo_event_{i}.json")
        event_file = open(os.path.join("../DB/raw", f"velo_event_{i}.json"))
        json_data = json.loads(event_file.read())
        events.append(json_data)
        event_file.close()

Loading file: velo_event_0.json
Loading file: velo_event_1.json
Loading file: velo_event_2.json
Loading file: velo_event_3.json
Loading file: velo_event_4.json
Loading file: velo_event_5.json
Loading file: velo_event_6.json
Loading file: velo_event_7.json
Loading file: velo_event_8.json
Loading file: velo_event_9.json
Loading file: velo_event_10.json
Loading file: velo_event_11.json
Loading file: velo_event_12.json
Loading file: velo_event_13.json
Loading file: velo_event_14.json
Loading file: velo_event_15.json
Loading file: velo_event_16.json
Loading file: velo_event_17.json
Loading file: velo_event_18.json
Loading file: velo_event_19.json
Loading file: velo_event_20.json
Loading file: velo_event_21.json
Loading file: velo_event_22.json
Loading file: velo_event_23.json
Loading file: velo_event_24.json
Loading file: velo_event_25.json
Loading file: velo_event_26.json
Loading file: velo_event_27.json
Loading file: velo_event_28.json
Loading file: velo_event_29.json
Loading file: velo_e

In [47]:
pipeline = TrackFollowingPipeline(events=events, intra_node=False)

In [ ]:
Optimiser = BayesianOptimiser(learning_rate=0.05)
optimal_parameters = pipeline.optimise_parameters(Optimiser, max_runs=10) # DO NOT remove max_runs, chances are that this will run forever (NO, I do what I want :-|)

Initial parameter set: {'x_slope': 0.8987096821652251, 'y_slope': 0.033896254633258294, 'x_tol': 0.7361048489303326, 'y_tol': 0.4729339581645684, 'scatter': 0.7204142797211387}


Optimising:  10%|█         | 1/10 [00:01<00:15,  1.71s/it]

temp param space: [array([0.10294941, 0.93376523, 0.87295784, 0.96052905, 0.82218285,
       0.1960673 , 0.41708212, 0.53447607, 0.73034143, 0.05159812]), array([0.12500998, 0.59351614, 0.88194581, 0.39564965, 0.84089879,
       0.83463086, 0.79335354, 0.8900538 , 0.33454592, 0.59847425]), array([0.47909861, 0.66226974, 0.68660045, 0.66295235, 0.79250916,
       0.4915044 , 0.56536541, 0.54713049, 0.78400457, 0.79531387]), array([0.56129037, 0.56647853, 0.45279764, 0.49562295, 0.46006599,
       0.57819637, 0.53032054, 0.43159728, 0.72074206, 0.46357409]), array([0.49622229, 0.03119338, 0.67390927, 0.10020481, 0.1478961 ,
       0.75185622, 0.39183346, 0.37975827, 0.37326952, 0.17565408])]


Optimising:  20%|██        | 2/10 [00:07<00:32,  4.02s/it]

Next parameter set: {'x_slope': np.float64(0.1029494136693857), 'y_slope': np.float64(0.12500997576416262), 'x_tol': np.float64(0.4790986123063894), 'y_tol': np.float64(0.5612903708584694), 'scatter': np.float64(0.49622228878509855)}
temp param space: [array([0.4876346 , 0.48698218, 0.04788626, 0.10157675, 0.74959094,
       0.0610747 , 0.55614315, 0.41637582, 0.29658253, 0.91737777]), array([0.59772517, 0.54240213, 0.85862571, 0.16068373, 0.89264334,
       0.50956637, 0.7742638 , 0.49504112, 0.03854242, 0.51197545]), array([0.44880547, 0.7019586 , 0.6825078 , 0.47583991, 0.45518057,
       0.46968782, 0.6815659 , 0.44241933, 0.75221368, 0.44165571]), array([0.75668907, 0.43302103, 0.44791308, 0.71052741, 0.75341333,
       0.41623799, 0.66062271, 0.72972518, 0.46702159, 0.6941013 ]), array([0.19737345, 0.34059784, 0.38714406, 0.11740354, 0.13200029,
       0.352487  , 0.62338231, 0.25921202, 0.51742836, 0.67197155])]


...

Optimising:  80%|████████  | 8/10 [01:11<00:20, 10.40s/it]/home/neto/Desktop/UM/project 2/velopix_tracking/venv/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 0.0001. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


temp param space: [array([0.15214981, 0.83803853, 0.91548076, 0.98320918, 0.58153777,
       0.86160877, 0.93316835, 0.48523419, 0.98219297, 0.34922301]), array([0.17515745, 0.07971492, 0.14881747, 0.4224034 , 0.87952346,
       0.79743446, 0.96417609, 0.48542345, 0.82043733, 0.11614147]), array([0.57918183, 0.73211617, 0.65933557, 0.63817803, 0.78733189,
       0.48507333, 0.45000735, 0.5699699 , 0.50270342, 0.48723498]), array([0.75991008, 0.45308814, 0.70783704, 0.67729064, 0.44663726,
       0.52014743, 0.68327734, 0.54641254, 0.79449126, 0.42361425]), array([0.72663266, 0.60858213, 0.49234288, 0.30891767, 0.26902738,
       0.31802187, 0.40760574, 0.70238572, 0.03597058, 0.32718094])]
Next parameter set: {'x_slope': np.float64(0.15214981415729267), 'y_slope': np.float64(0.1488174662652645), 'x_tol': np.float64(0.4850733319221243), 'y_tol': np.float64(0.5464125432909537), 'scatter': np.float64(0.4923428835581784)}


Optimising:  90%|█████████ | 9/10 [01:23<00:10, 10.76s/it]/home/neto/Desktop/UM/project 2/velopix_tracking/venv/lib/python3.12/site-packages/sklearn/gaussian_process/kernels.py:442: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 0.0001. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


temp param space: [array([0.08737838, 0.09482659, 0.16893636, 0.68714812, 0.55424484,
       0.88272112, 0.76076961, 0.17596076, 0.04449686, 0.67515209]), array([0.62535863, 0.49812935, 0.05244901, 0.18940221, 0.63113153,
       0.87745804, 0.88160742, 0.86999726, 0.6468682 , 0.04525353]), array([0.50646972, 0.51916475, 0.65899452, 0.75219136, 0.78153679,
       0.7358726 , 0.78767206, 0.58345812, 0.66374504, 0.56933185]), array([0.77480629, 0.59291322, 0.5352038 , 0.6658954 , 0.66158216,
       0.58101989, 0.47415614, 0.51985847, 0.57153685, 0.53666315]), array([0.39755911, 0.77921944, 0.10086065, 0.33015582, 0.29713793,
       0.469675  , 0.41775847, 0.43069428, 0.02874346, 0.68378135])]
Next parameter set: {'x_slope': np.float64(0.8827211245030829), 'y_slope': np.float64(0.04525353082882899), 'x_tol': np.float64(0.7358725962058361), 'y_tol': np.float64(0.47415613586411487), 'scatter': np.float64(0.6837813478942689)}


Optimising: 100%|██████████| 10/10 [01:33<00:00,  9.38s/it]


In [50]:
print(optimal_parameters) # Note these are just here for example...

{}
